In [ ]:
import scanpy as sc

import anndata2ri
import logging
from scipy.sparse import issparse
# from CSCORE.CSCORE_IRLS import CSCORE_IRLS
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

In [ ]:
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

In [ ]:
%%R
library(SPARK)
library(Seurat)

In [ ]:
adata = sc.read_h5ad("spatial_heart_visium_processed.h5ad")

In [ ]:
if issparse(adata.X):
    if not adata.X.has_sorted_indices:
        adata.X.sort_indices()
ro.globalenv["adata"] = adata

In [ ]:
%%R
seurat_obj = as.Seurat(adata, counts="X", data = NULL)
seurat_obj = RenameAssays(seurat_obj, originalexp = "RNA")

In [ ]:
%%R
rawcount = seurat_obj@assays$RNA@counts
info = data.frame(adata@int_colData$reducedDims$spatial)
colnames(rawcount) <- rownames(info)

In [ ]:
%%R 
location        <- as.matrix(info)
mt_idx      <- grep("mt-",rownames(rawcount))
if(length(mt_idx)!=0){
    rawcount    <- rawcount[-mt_idx,]
}

In [ ]:
%%R 
sparkX <- sparkx(rawcount,location,numCores=8,option="mixture")

In [ ]:
%%R 
str(sparkX$res_mtest)

In [ ]:
%%R
saveRDS(sparkX, "sparkresult.rds")

In [ ]:
select_gene = ro.r("sparkX$res_mtest")

In [ ]:
rank_gene = select_gene.sort_values(by = "adjustedPval")

In [ ]:
rank_gene.to_csv("rankgenelist.csv")